# Step 6: Experimental Deep Learning Model (PyTorch)

This notebook explores a simple neural network implemented in PyTorch
to forecast daily retail demand. The model is experimental and is used
to compare deep learning performance against Spark-based models.


Load Feature Data

In [ ]:
import pandas as pd

df = pd.read_parquet("../data/processed/daily_features")
df.head()


Select Features & Target

In [ ]:
feature_cols = [
    "lag_1",
    "lag_7",
    "rolling_7",
    "rolling_14",
    "day_of_week",
    "week_of_year",
    "month"
]

X = df[feature_cols].values
y = df["daily_quantity"].values


Train/Test Split

In [ ]:
split_index = int(len(df) * 0.8)

X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]


Convert to PyTorch Tensors

In [ ]:
import torch

X_train_t = torch.tensor(X_train, dtype=torch.float32)
X_test_t  = torch.tensor(X_test, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_t  = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)


Define Neural Network

In [ ]:
import torch.nn as nn

class DemandForecastNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        return self.fc2(x)

model = DemandForecastNet(X_train_t.shape[1])
model


Training Setup

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


Train the Model

In [ ]:
epochs = 50

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    outputs = model(X_train_t)
    loss = criterion(outputs, y_train_t)

    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.2f}")


Evaluate PyTorch Model

In [ ]:
model.eval()
with torch.no_grad():
    y_pred_torch = model(X_test_t)

y_pred_torch = y_pred_torch.numpy().flatten()


PyTorch Metrics (RMSE & MAE)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

rmse_torch = np.sqrt(mean_squared_error(y_test, y_pred_torch))
mae_torch = mean_absolute_error(y_test, y_pred_torch)

rmse_torch, mae_torch


Save PyTorch Predictions

In [ ]:
torch_results = pd.DataFrame({
    "actual": y_test,
    "predicted": y_pred_torch
})

torch_results.to_csv(
    "../data/processed/pytorch_predictions.csv",
    index=False
)

torch_results.head()


## PyTorch Model Interpretation

The PyTorch neural network was able to learn demand patterns but did not
significantly outperform Spark-based models.

Given the structured nature of the features and limited dataset size,
traditional regression models performed competitively.

This experiment highlights that deep learning is not always superior
for structured time-series demand forecasting problems.
